In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

In [18]:
path_prefix = '/home/joeyresuento/Projects/data_training/data_sets/ml-25m'
# movies = pd.read_csv(f'{path_prefix}/movies.csv')
# ratings = pd.read_csv(f'{path_prefix}/ratings.csv')

In [19]:
spark = SparkSession \
    .builder \
    .appName("MyApp") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

In [20]:
movies_df = spark.read.option("header", True).csv(f'{path_prefix}/movies.csv')
ratings_df = spark.read.option("header", True).csv(f'{path_prefix}/ratings.csv')

In [22]:
ratings_df.createOrReplaceTempView('ratings')
movies_df.createOrReplaceTempView('movies')
# spark.sql("SELECT COUNT(1) FROM movies").show()
# spark.sql("SELECT COUNT(1) FROM ratings").show()

In [23]:
top100_highest_rated_movies = spark.sql("SELECT \
    movieId, rating, COUNT(1) as count \
    FROM ratings \
    GROUP BY movieId, rating \
    ORDER BY rating, count DESC LIMIT 100")

In [24]:
top100_highest_rated_movie_ids = top100_highest_rated_movies \
    .rdd.map(lambda r: int(r.movieId)) \
    .collect()

In [29]:
movies_df.printSchema()
ratings_df.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [30]:
movie_ratings_df = spark.sql("SELECT m.movieId, m.title, m.genres, r.userId, r.rating, r.timestamp \
FROM movies m INNER JOIN ratings r ON m.movieId = r.movieId")

In [35]:
movie_ratings_df.rdd.map()

[Row(movieId='296', title='Pulp Fiction (1994)', genres='Comedy|Crime|Drama|Thriller', userId='1', rating='5.0', timestamp='1147880044'),
 Row(movieId='306', title='Three Colors: Red (Trois couleurs: Rouge) (1994)', genres='Drama', userId='1', rating='3.5', timestamp='1147868817'),
 Row(movieId='307', title='Three Colors: Blue (Trois couleurs: Bleu) (1993)', genres='Drama', userId='1', rating='5.0', timestamp='1147868828'),
 Row(movieId='665', title='Underground (1995)', genres='Comedy|Drama|War', userId='1', rating='5.0', timestamp='1147878820'),
 Row(movieId='899', title="Singin' in the Rain (1952)", genres='Comedy|Musical|Romance', userId='1', rating='3.5', timestamp='1147868510'),
 Row(movieId='1088', title='Dirty Dancing (1987)', genres='Drama|Musical|Romance', userId='1', rating='4.0', timestamp='1147868495'),
 Row(movieId='1175', title='Delicatessen (1991)', genres='Comedy|Drama|Romance', userId='1', rating='3.5', timestamp='1147868826'),
 Row(movieId='1217', title='Ran (1985)',